# Core Gene AMR Models

Este notebook é a reprodução do seguinte repositório:

https://github.com/jimdavis1/Core-Gene-AMR-Models

Aqui encontram-se os scripts necessários para construir o k-mer e modelos baseados em alinhamento para:
- Klebsiella
- Mycobacterium
- Salmonella
- Staphylococcus

## Diretórios

Automate: script que automatiza o treino para Klebsiella, Mycobacterium, Salmonella, and Staphylococcus datasets.

GenomicModelCreator: treino dos modelos e execução das estatísticas e métricas de um modelo usando dados do genoma e metadados. 

Subtree-Analysis: análise das árvores baseado no alinhamento de genes conservados.

Scripts: scripts específicos para criar alinhamentos one-hot makeAlignments.py.
makeMatrix.py pode ser usado para criar uma matriz, porém, GenomicModelCreator  faz isso por si só inicialmente, assim esse script é ignorado.

## Dados

Os dados foram obtidos de ftp://ftp.patricbrc.org/datasets/. Colocar em MeusProjetos

Na pasta, executar no cmd:

git submodule init <br>
git submodule update

Na pasta há quatro diretórios: Klebsiella, Mycobacterium, Salmonella, and Staphylococcus. A estrutura de cada diretório pode ser descrita como:

- clades_use : clados usados para gerar modelos baseados na filogenia
- fasta.X.Y : X representa o número de genes escolhidos para o conjunto de genes, Y é a replicata de uma amostra (existem múltiplas replicatas para um conjunto de 100 genes). Estes são os mesmos conjuntos usados no paper. Haverá um arquivo fasta por genoma nomeados com a convenção genome_id.fasta.
- fasta.low10 : os modelos foram criados usando os 10 conjuntos de 100 genes e os 10 genes com a menor importância foram escolhidos. Estes são esses genes e os arquivos fasta correspondentes por genoma. Haverá um arquivo fasta por genoma nomeados com a convenção genome_id.fasta.
- *.sir.filt.plf.tab : são os metadados brutos de AMR para cada espécie.


Obs: os diretórios contendo os arquivos fasta podem não conter o mesmo conjunto de genomas, porque genomas que não contêm todos os X genes maiores que metade do tamanho médio e menores que duas vezes o tamanho médio foram removidos.

## Automate.sh

Este script serve para automatizar a construção dos modelos baseados em alinhamento a partir dos alinhamentos do repositório. Utiliza o submódulo GenomicModelCreator.

O submódulo GenomicModelCreator necessita de alguns requerimentos:

- KMC deve estar instalado e os executáveis kmc, kmc_dump e kmc_tools devem estar no caminho.

https://sun.aei.polsl.pl//REFRESH/index.php?page=projects&project=kmc&subpage=download


- Python 2.7 e pacotes devem ser instalados: numpy, sklearn, scipy, xgboost. 

- Garanta que o diretório KMC em GenomicModelCreator esteja nos caminhos. Ele só aparece com a inicialização e atualização dos submódulos

### Executando:

Com tudo pronto e trabalhando na pasta MeusProjetos, rodar no bash:

bash /pathtoautomate/automate.sh /Modelo \<threads\>
    
threads é um argumento opcional onde definimos o número de threads. o default é 1.

Como já temos os dados, as linhas 51-54 estão comentadas:

echo "Downloading data..." <br>
curl ftp://ftp.patricbrc.org/datasets/Nguyen_et_al_2020.tar.gz > oDir/Nguyen_et_al_2020.tar.gz <br>
echo "Decompressing data..." <br>
tar -xzf oDir/Nguyen_et_al_2020.tar.gz -C oDir <br>

Então serão construídos vários tipos de modelos para Staphylococcus (Klebsiella, Mycobacterium, and Salmonella também podem ser adicionadas, no entanto). Esses modelos serão armazenados dentro de pastas de modelos, na pasta Modelo. 

Se quiser executar para além de Staphylococcus pode editar a primeira linha dos loops 'for', sendo elas 78, 99, 120, 142 e 163, para for i in (echo Klebsiella Mycobacterium Salmonella Staphylococcus); faça isso em vez de for i in (echo Staphylococcus); ou o subconjunto que desejar. Note que Klebsiella, Mycobacterium, Salmonella, and Staphylococcus são as únicas 4 espécies para as quais os modelos podem ser construídos.


Os seguintes tipos de modelo serão construídos:

- Alignment-based models
- Clade-weighted models (no weight, weight by clade size, weight by S/R distribution in clade, weight by both)
- Gene sets (10 replicates of 100 genes, 1 replicate of 25, 50, 250, and 500 genes)
- Shuffled labels (10 replicates of 100 genes)


Script Output

no diretório de saída teremos:

models Directory: 1 diretório por espécie contendo o modelo de cada uma. <br>
Nguyen_et_al_2020 Directory: dados brutos para rodar o modelo. Esses dados são equivalentes aos usados no paper. <br>
Nguyen_et_al_2020.tar.gz File: Tarball o qual foi baixado e extraído do diretorio Nguyen_et_al_2020. <br>
rand.tab File: Gerado em tempo real, embaralhamento aleatório do modelo de rótulos embaralhados da última execução. <br>
temp Directory: diretório para armazenar temporariamente os dados durante a execução. Pode ser apagado. <br>

O diretório dos modelos terá um diretório para cada espécie, cada qual contendo múltiplos modelos com a respectiva estrutura de pastas:

all : diretório contendo 5 de 10 folds run. Cada fold contém um .pred, .train_hist.txt, .true, .tree, e pkl representand os rótulos preditivos para o conjunto de teste, história do treino (merror for train, validation, and test), valores verdadeiros para o conjunto de teste, representação em texto da tree ensemble, e o modelo propriamente dito em formato pkl. Adicionalmente, arquivos com métricas estatísticas também estão incluídos: confusion_matrix.tab, f1.tab, raw_acc.tab, e VMEME.tab. model.params é incluído para os parâmetros do XGBoost usados no modelo.
model.attrOrder : atribui ordem para os features do modelo.
model.genomes.list : lista dos genomas usados para treinar o modelo.
model.labels.map : label mapping
model.params : parâmetros enviados ao script do modelo
temp.txt : temprary file (used for debugging)
weights.list : lista de pesos (used for debugging)

In the event you run into an out of memory error of any sort, it's likely that the machine being used lacks the RAM to run a certain model.

Also note that the training script uses the /dev/shm/ directory to store KMC runs. This directory should be initialized and writable or the script may fail to run! The link for downloading KMC is in the Readme file for the Automate subdirectory.

## Subtree-Analysis

Agora, veremos como as subárvores são construídas em *Predicting antimicrobial resistance using conserved genes* by Nguyen et al. 

Para determinar se a diversidade de cepas teve influência nos modelos de previsão do fenótipo AMR, construímos árvores filogenéticas e normalizamos a contribuição de cada genoma para cada modelo com base no tamanho da subárvore de onde veio, bem como na distribuição de suscetibilidade e fenótipos resistentes para os genomas dentro de cada subárvore. Os modelos foram então construídos para subárvores definidas em diferentes distâncias das árvores.

